In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import brownian_motion as bm 
from scipy.stats import jarque_bera
from scipy.stats import norm
import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import yfinance as yf
from pandas_datareader import data as pdr
from datetime import datetime

In [ ]:
N = norm.cdf
kappa =4
theta = 0.02
v_0 =  0.02
xi = 0.9
r = 0.02
S = 100
paths =50000
steps = 2000
T = 1





def get_Heston_matr(S0:float, v0:float , n: int, d: int, alpha:float, theta:float, sigma:float, rho, dt=1):
    """Retornar d trayectorias de tamaño n de Movimiento browniano geométrico con volatilidad estocástica
    Recibe: 
    
    Retorna:
    S: matriz de trayectorias de precios
    V: matriz de trayectorias de volatilidades. Cada fila de V es la volatilidad de la fila correspondiente en S
    """
    if v0<0:
        raise Exception("Condiciones iniciales de volatilidad negativas")
    S=np.zeros((n,d))
    for i in range(d):
        B1,B_=bm.get_B(n,dt=dt),#Brownianos indep
        B2=rho*B1+np.sqrt(1-rho**2)*B_#Dos Movimientos brownianos correlacionados
        dB1=bm.dif_B(B1)# Devuelve el vector de incrementos agregando 0 al inicio para tener igual longitud
        dB2=bm.dif_B(B2)
        

    return None





def generate_heston_paths(S, T, r, kappa, theta, v_0, rho, xi, 
                          steps, Npaths, return_vol=False):
    dt = T/steps
    size = (Npaths, steps)
    prices = np.zeros(size)
    sigs = np.zeros(size)
    S_t = S
    v_t = v_0
    for t in range(steps):
        WT = np.random.multivariate_normal(np.array([0,0]), 
                                           cov = np.array([[1,rho],
                                                          [rho,1]]), 
                                           size=paths) * np.sqrt(dt) 
        
        S_t = S_t*(np.exp( (r- 0.5*v_t)*dt+ np.sqrt(v_t) *WT[:,0] ) ) 
        v_t = np.abs(v_t + kappa*(theta-v_t)*dt + xi*np.sqrt(v_t)*WT[:,1])
        prices[:, t] = S_t
        sigs[:, t] = v_t
    
    if return_vol:
        return prices, sigs